In [1]:
# Math
import numpy as np
import scipy.sparse.linalg
from scipy import optimize


### GL-SigRep

#### Inputs : 
- Input signal X : X
- number of iterations iter
- parameters α, β

In [2]:
# test variables to run code

m = 10
n = 10
alpha = 0.1
beta = 0.1

x = np.random.rand(m,n)
l = np.random.rand(m,m)

In [3]:
max_iter = 5

y = x

In [4]:
# minimize L s.t alpha * trace(Y.T * L * Y) + beta * (norm_F(L))^2 s.t trace(L) = n; L_ij = L_ji <= 0; L * ones = zeros
def con_sum_zero(l):
    l2 = l.reshape(m,m)
    return l2.sum(axis=0)

def con_trace(l):
    l2 = l.reshape(m,m)
    return np.trace(l2) - m

def con_sym(l):
    l2 = l.reshape(m,m)
    return ((l2 - l2.T)**2).sum()

cons = [{'type':'eq', 'fun': con_sum_zero},
        {'type':'eq', 'fun': con_trace},
        {'type':'eq', 'fun': con_sym}]

def func_1(l, alpha, beta):
    l2 = l.reshape(m, m)
    return alpha * np.trace(y.T * l2 * y) + beta * (l2**2).sum()

# minimize Y s.t norm_F(X-Y)^2 + alpha * trace(Y.T * L * Y)
def func_2(y, alpha):
    y2 = y.reshape(m, n)
    return ((x-y2)**2).sum() + alpha * np.trace(y2.T * l * y2)


In [5]:
for t in range(max_iter):   
    # step to update Graph Laplacian
    res = optimize.minimize(func_1, l.reshape(m*m,1), args=(alpha, beta), constraints=cons)
    l = res.x.reshape(m,m)
    
    # step to update Y
    res = optimize.minimize(func_2, y.reshape(m*n,1), args=(alpha,))
    y = res.x.reshape(m,n)
    

In [6]:
np.trace(l)

10.0

In [7]:
((l - l.T)**2).sum()

7.9912396660829535e-07

In [8]:
l.sum(axis=0)

array([  2.77555756e-17,  -1.94289029e-16,   1.38777878e-16,
        -8.32667268e-17,  -1.38777878e-17,   1.11022302e-16,
         5.55111512e-17,  -9.71445147e-17,   3.88578059e-16,
         0.00000000e+00])

In [9]:
(l[np.where(~np.eye(l.shape[0],dtype=bool))] > 0).sum()

0

In [10]:
l

array([[ 0.98535916, -0.1174943 , -0.10765729, -0.11576445, -0.09477542,
        -0.11614143, -0.09040606, -0.1068269 , -0.11744312, -0.11897343],
       [-0.11742495,  1.06564124, -0.11761711, -0.12572303, -0.10473696,
        -0.1261016 , -0.1003665 , -0.11678781, -0.1274026 , -0.12893268],
       [-0.10765567, -0.11768252,  0.98688705, -0.11595303, -0.09496741,
        -0.11633053, -0.09059645, -0.10701805, -0.11763192, -0.11916198],
       [-0.11570725, -0.12573527, -0.11589903,  1.0517931 , -0.10301852,
        -0.12438271, -0.09864782, -0.11507004, -0.1256853 , -0.12721511],
       [-0.0948624 , -0.10488749, -0.09505078, -0.10315837,  0.88375689,
        -0.1035359 , -0.07780116, -0.09422328, -0.10483827, -0.10636868],
       [-0.11608293, -0.12610935, -0.11627422, -0.12438074, -0.10339369,
         1.05481645, -0.09902254, -0.11544441, -0.12606034, -0.12759099],
       [-0.09052037, -0.10054652, -0.09071033, -0.09881768, -0.07783045,
        -0.09919565,  0.8487658 , -0.08988078